Used with release-04-02-09

In [15]:
import os
import basf2
import modularAnalysis as ma
from variables import variables

In [16]:
main = basf2.create_path()

In [17]:
sample = "charged"
jobNo = 1
trackSelection = "user"
jobFileSize = 1

In [18]:
######################################################
#
# 4. Reading in data files
#
# #####################################################

mc_path = "/ghi/fs01/belle2/bdata/MC/MC13a_cdst/generic/"
filler = "_cdst_eph3_" 

if sample == 'signal':
    ma.inputMdstList('default', ['/group/belle2/tutorial/2019_June/Bu2JpsiK/mdst_Bu2JpsiK_*.root'], path=main)

elif sample in ['charged', 'mixed', 'uubar', 'ddbar', 'ssbar', 'ccbar']:
    file_folder = mc_path + sample + "/"
    input_file_list =  [file_folder + sample + filler + "%d.root" %i for i in range(jobNo*jobFileSize, (jobNo+1)*jobFileSize)]
    print("\nInput file list:\n", input_file_list, "\n")
elif sample == 'proc10':
    ma.inputMdstList('default', 
                     ['/group/belle2/dataprod/Data/OfficialReco/proc10/e0007/4S/GoodRuns/r*/skim/hlt_hadron/mdst/sub00/*.root',
                      '/group/belle2/dataprod/Data/OfficialReco/proc10/e0008/4S/GoodRuns/r*/skim/hlt_hadron/mdst/sub00/*.root'],
                     path=main)
else:
    print('[ERROR] The only valid sample names are charged, mixed, uubar, ddbar, ssbar, ccbar. Needs fixing: signal and proc10!')
    
ma.inputMdstList("default", input_file_list, path=main)



Input file list:
 ['/ghi/fs01/belle2/bdata/MC/MC13a_cdst/generic/charged/charged_cdst_eph3_1.root'] 



In [19]:
from variables import variables as vm
# Aliases
vm.addAlias("eID", "electronID")
vm.addAlias("piID", "pionID")
vm.addAlias('R_Kpi', 'pidPairProbabilityExpert(321, 211, ARICH,TOP,CDC)')  # kaon/pion w/o arich
vm.addAlias("muID", "muonID")
vm.addAlias("D_p_cms", "useCMSFrame(p)")

True

In [20]:
######################################################
#
# 6. Creating and filling final state ParticleLists
#
# #####################################################
from stdCharged import stdMu, stdPi

# basic quality cut strings
trackQuality = 'thetaInCDCAcceptance'
ipCut = 'dr < 0.5 and abs(dz) < 2.0'
muPidCut  = 'muID > 0.9'
piPidCut = 'R_Kpi < 0.9'
userCut = trackQuality + ' and ' + ipCut

# create and fill final state ParticleLists
if trackSelection == 'MCmatch':
    if sample == 'proc10':
        print('You can not select tracks based on MC information when running over real data!')
elif trackSelection == 'loose':
    stdMu('loose', path=main)
    stdPi('loose', path=main)
elif trackSelection == 'good':
    stdMu('good', path=main)
    stdPi('good', path=main)
elif trackSelection == 'user':
    ma.fillParticleList('mu+:user', userCut + ' and ' + muPidCut, path=main)
    ma.fillParticleList('pi+:user', userCut + ' and ' + piPidCut, path=main)
else:
    print('For the track selection the only valid options so far are MCmatch, loose, good, user!')

In [21]:
######################################################
#
# 7. Reconstructing phi from mumu-pair
#
# #####################################################
phi_mass_window = '0.99 < M < 1.05'
phi_cuts = phi_mass_window #+ ' and ' + jpsipstarwindow

#ma.reconstructDecay('phi:mumu -> mu+:'+trackSelection + ' mu-:'+trackSelection, cut=phi_cuts, path=main)


In [22]:
######################################################
#
# 8. Reconstructing D+ from pi+ mu+ mu+
#
# #####################################################
dmasswindow = '1.70 < M < 2.1' #mass = 1.8696 GeV
number_of_tracks = "nTracks > 2"
D_cms_p = "D_p_cms > 2.5"
d_cuts = dmasswindow + ' and ' + number_of_tracks + ' and ' + D_cms_p #+ ' and ' + jpsipstarwindow

ma.reconstructDecay('D+:mumu -> mu+:'+trackSelection + ' mu-:'+trackSelection + ' pi+:' + trackSelection, cut=d_cuts, path=main)


In [23]:
######################################################
#
# 9. Fitting the B+ decay vertex
#
# #####################################################
from vertex import vertexKFit, vertexTree
vertexKFit("D+:mumu", 0.000, path=main)

In [24]:
######################################################
#
# 12. Performing MC matching (MC truth association)
#
# #####################################################

if(sample != 'proc11'):
    #ma.matchMCTruth("phi:mumu", path=main)
    ma.matchMCTruth("D+:mumu", path=main)

In [25]:
######################################################
#
# 13. Creating list of variables for ntuple
#
# #####################################################
import variables.collections as vc
import variables.utils as vu


std_vars = vc.mc_truth + vc.kinematics

# Variables for final states (electrons, positrons, pions)
fs_vars = vc.pid + std_vars #vc.track + vc.track_hits

cmskinematics = vu.create_aliases(
    vc.kinematics, "useCMSFrame({variable})", "CMS")

if(sample != 'proc11'):
    std_vars += vc.mc_kinematics + vc.mc_truth

d_vars = []

d_vars += std_vars
d_vars += vc.inv_mass
d_vars += cmskinematics
d_vars += ["pValue"]

# Variables for final states (electrons, positrons, pions)
fs_vars = vc.pid + ["R_Kpi"] + std_vars + vc.inv_mass#vc.track + vc.track_hits

d_vars += vu.create_aliases_for_selected(

    fs_vars,

    "D+:mumu -> ^mu+:"+trackSelection+" ^mu-:"+trackSelection + "^pi+:" + trackSelection,

    prefix=["mu_p", "mu_m", "pi_p"],

)

In [26]:
"""phi_vars = []
phi_vars += std_vars
phi_vars += vc.inv_mass

phi_vars += vu.create_aliases_for_selected(

    fs_vars,

    "phi:mumu -> ^mu+:"+trackSelection+" ^mu-:"+trackSelection,

    prefix=["mup", "mum"],

)"""

'phi_vars = []\nphi_vars += std_vars\nphi_vars += vc.inv_mass\n\nphi_vars += vu.create_aliases_for_selected(\n\n    fs_vars,\n\n    "phi:mumu -> ^mu+:"+trackSelection+" ^mu-:"+trackSelection,\n\n    prefix=["mup", "mum"],\n\n)'

In [27]:
######################################################
#
# 14. Writing ntuple to output file
#
# #####################################################
outDir = 'recodata'
if not os.path.exists(outDir):
    os.makedirs(outDir)

filenameEnd = '_'.join([sample,str(jobNo),str(jobFileSize),trackSelection]) + '.root'

outputFileName = outDir + '/d2pimumu_reco_' + filenameEnd

#main.add_module('VariablesToNtuple', particleList='phi:mumu', fileName=outputFileName, treeName='phi', variables=phi_vars)
main.add_module('VariablesToNtuple', particleList='D+:mumu', fileName=outputFileName, treeName='D', variables=d_vars)



In [ ]:
ma.summaryOfLists(['D+:mumu'], path=main)

In [28]:
basf2.process(main)

In [29]:
print(basf2.statistics)